In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [92.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-09-20 01:24:34--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.26MB/s    in 0.7s    

2022-09-20 01:24:36 (1.26 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ApparelCloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Apparel_v1_00.tsv.gz"), sep="\t", header=True)

df.show(6)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [6]:
df = df.dropna()
df.show(6)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [7]:
# REVIEW_ID TABLE

review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df = review_id_df.dropDuplicates(["review_id"])
review_id_df.show(6)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10002DNI29WUF|   10751858|B004DMYMTQ|     968881660| 2014-11-11|
|R10004URUIJHDJ|   18075456|B00JY1FYWU|     986738401| 2015-07-05|
|R1000E649H4MWR|   23790097|B00H57YK0W|      41473452| 2014-03-25|
|R1000HDDRRIGD1|   47091070|B00ABS67T4|     862126754| 2015-01-08|
|R1000OWRQKQWE2|   11513874|B00AZP006O|     357356824| 2015-06-01|
|R1000WY1C132ZU|   38458906|B00K1OWZS0|     535981341| 2014-10-06|
+--------------+-----------+----------+--------------+-----------+
only showing top 6 rows



In [8]:
from pyspark.sql.types import * 

review_id_df = review_id_df.withColumn("customer_id",review_id_df["customer_id"].cast(IntegerType()))\
    .withColumn("product_parent",review_id_df["product_parent"].cast(IntegerType()))\
    .withColumn("review_date",review_id_df["review_date"].cast(DateType()))

review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [9]:
# PRODUCTS TABLE

products_df = df.select(["product_id", "product_title"])
products_df = products_df.dropDuplicates(["product_id"])
products_df.show(6)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0000032034|Adult Ballet Tutu...|
|0000047228|Patriotic Stars &...|
|0615595154|Time You Can't Ge...|
|1465014578|2013 Desk Pad Cal...|
|1607056992|Eco Tote - Skinny...|
|1608322114|Bambino Balls Sho...|
+----------+--------------------+
only showing top 6 rows



In [10]:
# CUSTOMERS TABLE

customers_df = df.select(["customer_id"])
customers_df.show(6)

+-----------+
|customer_id|
+-----------+
|   32158956|
|    2714559|
|   12608825|
|   25482800|
|    9310286|
|   26631939|
+-----------+
only showing top 6 rows



In [11]:
customers_df = customers_df.groupBy("customer_id").count()
customers_df.orderBy("customer_id").select(["customer_id", "count"])
customers_df.show(6)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    1662872|   41|
|   51854170|    2|
|   24666070|    2|
|   18358679|    2|
|   17139768|    1|
|   17944190|    1|
+-----------+-----+
only showing top 6 rows



In [12]:
customers_df = customers_df.withColumnRenamed("count", "customer_count")
customers_df.show(6)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    1662872|            41|
|   51854170|             2|
|   24666070|             2|
|   18358679|             2|
|   17139768|             1|
|   17944190|             1|
+-----------+--------------+
only showing top 6 rows



In [13]:
# VINE TABLE

vine_df = df.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine"])
vine_df = vine_df.dropDuplicates(["review_id"])
vine_df.show(6)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10000QFNN3ENR|          4|            0|          0|   N|
|R10002DNI29WUF|          5|            0|          0|   N|
|R10004URUIJHDJ|          5|            0|          0|   N|
|R100098KCVN4CT|          5|            0|          0|   N|
|R1000AIZRA8TT7|          5|            0|          0|   N|
|R1000B3IIMFVCJ|          2|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 6 rows

